## Optimización de inventarios y fuerza de ventas

En este notebook presentamos optimizaciones dinámicas para la gestión de inventarios y la asignación de la fuerza de ventas en función de los patrones de ventas identificados en los datos.

## Clustering de productos

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv('/home/marcos/Desktop/Reto/data_clean/ventas_clean.csv')

### Feature engineering

In [51]:
# Semana, mes, día de la semana
df['Fecha'] = pd.to_datetime(df['Fecha'])

df['semana'] = df['Fecha'].dt.isocalendar().week
df['mes'] = df['Fecha'].dt.month
df['dia_semana'] = df['Fecha'].dt.dayofweek

# Agregamos columnas de autocorrelación por categoria y region

# Lag 1 (ventas del periodo anterior)
df['ventas_categoria_lag_1'] = df.groupby(['Categoria'])['Cantidad'].shift(1)
df['ventas_region_lag_1'] = df.groupby(['Region'])['Cantidad'].shift(1)

# Rolling Mean 3, 7, 14, 30 días
df['ventas_categoria_rolling_mean_3'] = df.groupby(['Categoria'])['Cantidad'].transform(lambda x: x.rolling(window=3).mean())
df['ventas_region_rolling_mean_3'] = df.groupby(['Region'])['Cantidad'].transform(lambda x: x.rolling(window=3).mean())

df['ventas_categoria_rolling_mean_7'] = df.groupby(['Categoria'])['Cantidad'].transform(lambda x: x.rolling(window=7).mean())
df['ventas_region_rolling_mean_7'] = df.groupby(['Region'])['Cantidad'].transform(lambda x: x.rolling(window=7).mean())

df['ventas_categoria_rolling_mean_14'] = df.groupby(['Categoria'])['Cantidad'].transform(lambda x: x.rolling(window=14).mean())
df['ventas_region_rolling_mean_14'] = df.groupby(['Region'])['Cantidad'].transform(lambda x: x.rolling(window=14).mean())

df['ventas_categoria_rolling_mean_30'] = df.groupby(['Categoria'])['Cantidad'].transform(lambda x: x.rolling(window=30).mean())
df['ventas_region_rolling_mean_30'] = df.groupby(['Region'])['Cantidad'].transform(lambda x: x.rolling(window=30).mean())

# Ordenamos
df = df.sort_values(by=['Categoria', 'Region', 'Fecha'])
# No rellenamos NaN porque planeamos usar modelos que los soporten

df.to_csv('../data_clean/data_fe.csv', index=False)

df.head()

,ID_Venta,Fecha,ID_Cliente,ID_Producto,Cantidad,Metodo_Pago_cat,Estado,Categoria,ID_Categoria,Precio_Unitario,...,ventas_categoria_lag_1,ventas_region_lag_1,ventas_categoria_rolling_mean_3,ventas_region_rolling_mean_3,ventas_categoria_rolling_mean_7,ventas_region_rolling_mean_7,ventas_categoria_rolling_mean_14,ventas_region_rolling_mean_14,ventas_categoria_rolling_mean_30,ventas_region_rolling_mean_30
1029,1449,2024-02-05,172,29,6,2,Completa,Bebidas,6,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,2043,2024-02-08,318,27,2,5,Completa,Bebidas,6,6.21,...,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185,1680,2024-02-11,241,28,3,3,Completa,Bebidas,6,9.45,...,2.0,2.0,3.666667,3.666667,NaN,NaN,NaN,NaN,NaN,NaN
1235,1754,2024-02-11,212,27,3,5,Completa,Bebidas,6,6.21,...,3.0,3.0,2.666667,2.666667,NaN,NaN,NaN,NaN,NaN,NaN
1923,2703,2024-02-11,255,29,3,1,Pendiente,Bebidas,6,11.54,...,3.0,3.0,3.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN
